In [1]:
import pandas as pd
import etltools.etltools as etl

In [2]:
#!pip install ./dist/etltools-0.1.0-py3-none-any.whl

In [3]:
ejecutor = etl.etljob('docker','docker','postgres','5432')

In [4]:
ejecutor.etl()

extract...
https://api.covidtracking.com/v1/states/daily.csv
        date state  positive  deathConfirmed
0 2021-03-07    AK   56886.0             NaN


In [5]:
!cat app.log

2023-02-20 23:27:00,489 - root - DEBUG - >>> Running 'connect_db' 
2023-02-20 23:27:00,543 - root - DEBUG - >>> Executed 'connect_db' 
2023-02-20 23:27:00,543 - root - DEBUG - >>> Running 'extract' 
2023-02-20 23:27:03,476 - root - DEBUG - >>> Executed 'extract' 
2023-02-20 23:27:03,476 - root - DEBUG - >>> Running 'transform' 
2023-02-20 23:27:03,481 - root - DEBUG - >>> Executed 'transform' 
2023-02-20 23:27:03,482 - root - DEBUG - >>> Running 'load_data_to_db' 
2023-02-20 23:27:05,878 - root - DEBUG - >>> Executed 'load_data_to_db' 
2023-02-20 23:27:05,878 - root - DEBUG - >>> Running 'load_data_to_db' 
2023-02-20 23:27:06,033 - root - DEBUG - >>> Executed 'load_data_to_db' 


In [ ]:
# un truco para optimizar el flujo

def read_sql_tmpfile(query, db_engine):
    with tempfile.TemporaryFile() as tmpfile:
        copy_sql = "COPY ({query}) TO STDOUT WITH CSV {head}".format(
           query=query, head="HEADER"
        )
        conn = db_engine.raw_connection()
        cur = conn.cursor()
        cur.copy_expert(copy_sql, tmpfile)
        tmpfile.seek(0)
        df = pandas.read_csv(tmpfile)
        return df